In [1]:
import random
from bolib.Bo import Bo
from bolib.Dimension import NumericDimension
from bolib.ComputeSpace import ComputeSpace
from bolib.Normalizer import OptimizeFor
import pandas as pd
from IPython.display import clear_output
import numpy as np

c:\Users\phili\Documents\workspace\botorch\botorch_contrast\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
blackbox_dims = 10

In [3]:

def getRating(v, target):    
    return [sum([1 for i in v if i in target])]
    
def getSample(TARGET = None):
    s = [random.randint(i*10, i*10+9) for i in range(blackbox_dims)]

    if TARGET is not None:
        
        r= getRating(s, TARGET)
        return [s, r]
    return s


TARGET = getSample()

s, r = getSample(TARGET)
pd.DataFrame([[s, r, TARGET]], columns=["guess", "rating", "target"])  

,guess,rating,target
0,"[1, 13, 21, 30, 44, 50, 60, 72, 80, 91]",[0],"[3, 10, 28, 39, 45, 53, 61, 74, 84, 92]"


In [4]:
[ [i*10+j for j in range(10)] for i in range(blackbox_dims) ]
s = [random.randint(i*10, i*10+9) for i in range(blackbox_dims)]
s

[5, 16, 26, 33, 43, 58, 66, 72, 80, 97]

In [5]:
from bolib.Dimension import DiscreteDimension


numerics = [ DiscreteDimension(elements=[i*10+j for j in range(10)]  ,name=f"p{i}") for i in range(blackbox_dims) ]
ranking = [ NumericDimension(min=0, max=10, name="Ranking", optimize_for=OptimizeFor.MIN) ]



def infer(x, y, bo, cs):  
  cs.add_values(xs = [x], ys=[y])   

  inf = bo.infer()
  denorm = cs.denormalize(inf)
  return [d.value for d in denorm]





In [6]:
for i in numerics:
  print(i._elements)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
[40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
[60, 61, 62, 63, 64, 65, 66, 67, 68, 69]
[70, 71, 72, 73, 74, 75, 76, 77, 78, 79]
[80, 81, 82, 83, 84, 85, 86, 87, 88, 89]
[90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [7]:
compSpace = ComputeSpace(numerics, ranking)
bo = Bo(compSpace)

last_rating = 0
last_guess = None
best_guess = None
best_rating = None
print(f"We are looking for {TARGET}")

df_cols = ["guess", "rating", "target"]
df = pd.DataFrame([], columns=df_cols)  

for i in range(50):
  last_rating = r[0]
  last_guess = s
  
  
  guess = infer(s,r, bo, compSpace)  
  print("guess", guess)
  r = getRating(guess, TARGET)

  if r[0] == 0:
    guess = getSample()

  tdf = pd.DataFrame([[s, r, TARGET]], columns=df_cols)  
  
  df = pd.concat([df, tdf], axis=0).reset_index(drop=True)
  
  clear_output(wait=True)

  print(df)

  
  #if r[0] == 2 
  """
  if r[0] < 0.15:
    print(f"Found it in {i} steps: {guess}")
    break
  """
  if best_guess is None:
    best_rating = r[0]
    best_guess = guess

  if r[0] > best_rating:
    best_guess = guess
    best_rating = r[0]
  s = guess

print(f"I think it is {best_guess} with a rating of {best_rating}")


  


  



                                      guess rating  \
0   [5, 16, 26, 33, 43, 58, 66, 72, 80, 97]    [1]   
1   [9, 10, 22, 35, 47, 52, 63, 78, 89, 96]    [0]   
2   [3, 17, 24, 34, 43, 56, 65, 73, 89, 97]    [1]   
3   [4, 17, 25, 33, 43, 57, 66, 72, 84, 97]    [1]   
4   [1, 18, 23, 35, 43, 55, 64, 74, 89, 97]    [0]   
5   [2, 13, 27, 31, 44, 58, 65, 71, 82, 98]    [0]   
6   [7, 14, 22, 36, 42, 56, 61, 71, 83, 91]    [0]   
7   [1, 12, 21, 30, 45, 57, 62, 77, 83, 95]    [0]   
8   [6, 12, 22, 31, 48, 59, 68, 76, 84, 94]    [0]   
9   [6, 12, 21, 31, 42, 58, 69, 74, 81, 93]    [1]   
10  [5, 12, 22, 31, 45, 58, 67, 75, 82, 94]    [1]   
11  [4, 14, 28, 32, 43, 59, 65, 71, 80, 98]    [0]   
12  [9, 15, 22, 38, 41, 51, 61, 70, 84, 96]    [2]   
13  [8, 14, 22, 37, 41, 53, 61, 70, 83, 93]    [2]   
14  [0, 12, 22, 30, 45, 57, 61, 76, 83, 96]    [1]   
15  [7, 12, 21, 31, 40, 58, 69, 73, 80, 92]    [0]   
16  [2, 11, 26, 31, 41, 54, 63, 78, 86, 99]    [1]   
17  [3, 11, 27, 31, 40, 54, 

In [11]:
print(f"I think it is \n{best_guess} with a rating of {best_rating}")
print(TARGET)


I think it is 
[8, 14, 22, 37, 41, 53, 61, 70, 83, 93] with a rating of 2
[3, 10, 28, 39, 45, 53, 61, 74, 84, 92]
